In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import numpy as np
import os
import cv2
from tqdm import tqdm

In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
import tensorflow as tf

In [ ]:
from tensorflow.keras.applications.densenet import DenseNet169
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
DATADIR = r"/content/gdrive/My Drive/Weather"

CATEGORIES = ['Cloudy', 'Rain', 'sunny', 'sunset']

IMG_SIZE =150
img_rows, img_cols = 150,150
training_data = []

def create_training_data():
    for category in CATEGORIES:
        
        path = os.path.join(DATADIR,category)
        class_num = CATEGORIES.index(category)
        
        for img in tqdm(os.listdir(path)):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array = cv2.resize(img_array,(IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num]) 
            except Exception as e:
                pass

create_training_data()

print(len(training_data))

X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)



  0%|          | 0/300 [00:00<?, ?it/s]

  5%|▌         | 16/300 [00:00<00:01, 157.32it/s]

 10%|█         | 30/300 [00:00<00:01, 150.32it/s]

 13%|█▎        | 38/300 [00:05<00:50,  5.18it/s] 

 15%|█▍        | 44/300 [00:10<01:38,  2.59it/s]

 16%|█▌        | 48/300 [00:13<02:01,  2.07it/s]

 17%|█▋        | 51/300 [00:15<02:16,  1.82it/s]

 18%|█▊        | 53/300 [00:16<02:21,  1.74it/s]

 18%|█▊        | 55/300 [00:17<02:28,  1.65it/s]

 19%|█▊        | 56/300 [00:18<02:36,  1.55it/s]

 19%|█▉        | 57/300 [00:19<02:45,  1.47it/s]

 19%|█▉        | 58/300 [00:20<02:48,  1.44it/s]

 20%|█▉        | 59/300 [00:20<02:40,  1.50it/s]

 20%|██        | 60/300 [00:21<02:48,  1.42it/s]

 20%|██        | 61/300 [00:22<02:53,  1.38it/s]

 21%|██        | 62/300 [00:22<02:40,  1.49it/s]

 21%|██        | 63/300 [00:23<02:44,  1.44it/s]

 21%|██▏       | 64/300 [00:24<02:32,  1.55it/s]

 22%|██▏       | 65/300 [00:24<02:52,  1.36it/s]

 22%|██▏       | 66/300 [00:25<02:39,  1.46it/s]

 22%

1123


In [ ]:
X1 = np.array(X)
X1.shape

(1123, 150, 150, 3)

In [ ]:
y = np.array(y)
y.shape

(1123,)

In [ ]:

from sklearn.model_selection import train_test_split
from keras.utils import np_utils

X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.2, random_state=4)
X_train = X_train.reshape(X_train.shape[0],img_rows, img_cols,3)
X_test = X_test.reshape(X_test.shape[0],img_rows, img_cols,3)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

Y_train = np_utils.to_categorical(y_train, 4)
Y_test = np_utils.to_categorical(y_test, 4)





In [ ]:
X_test.shape

(225, 150, 150, 3)

In [ ]:
Y_train.shape

(898, 4)

In [ ]:
input_shape = [150, 150, 3]
model = Sequential()
model.add(VGG19(
    include_top=False,
    weights=None,
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=4,
    classifier_activation="softmax",
))
model.add(Flatten())
model.add(Dense(1024,activation="relu"))
model.add(Dense(units=4,activation="softmax"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 4, 4, 512)         20024384  
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              8389632   
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 4100      
Total params: 28,418,116
Trainable params: 28,418,116
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history=model.fit(X_train,Y_train,batch_size=32,epochs=10,verbose=1,validation_split=0.2)

Epoch 1/10
 1/23 [>.............................] - ETA: 11:38 - loss: 1.3771 - accuracy: 0.3750

KeyboardInterrupt: ignored

In [ ]:
scores = model.evaluate(X_train, Y_train, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

accuracy: 30.73%


In [ ]:
model.save("/content/gdrive/My Drive/model/weather_vgg19.h5")
print("Saved model to disk")

Saved model to disk
